Importation des packages

In [68]:
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO

Paramétrage de l'API

In [69]:
BASE_URL_API = "https://apidf-preprod.cerema.fr"
#code_insee, nom =  "59512", "Roubaix" #"75015", "Paris"
#code_insee Lille = 59350
code_insee = "78220"
url = BASE_URL_API + f"/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350"
# %% 
print(url)

https://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350


In [70]:
response = requests.get(url)#, params=params)
# Vérification du succès de la requête
if response.status_code == 200:
    data = response.json()  # On récupère les données au format JSON
else:
    print(f"Erreur : {response.status_code}")
# %%
print(type(data))
print(data.keys())

<class 'dict'>
dict_keys(['type', 'count', 'next', 'previous', 'features'])


Data est un dictionnaire ; on va transformer son entrée "features" en un dataframe python

In [72]:
# Conversion en DataFrame pandas
df = pd.DataFrame(data["features"])
print(df.head())
print(df.columns)

        id     type                                           geometry  \
0  8341428  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[3....   
1  8341455  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[3....   
2  8341456  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[3....   
3  8341460  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[3....   
4  8341463  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[3....   

                                          properties  
0  {'idmutinvar': '83a68c1f0c617ae2609e8ade54c118...  
1  {'idmutinvar': '45005cd8bafee591ed6debb57bbc66...  
2  {'idmutinvar': 'ebbcec3f97add3b68e7b452f8ba73f...  
3  {'idmutinvar': '7bcb49cdc14b9303042d3938d9ff75...  
4  {'idmutinvar': 'aee36b8e79a346304326d5bbed32e4...  
Index(['id', 'type', 'geometry', 'properties'], dtype='object')


In [73]:
df.head(1)

,id,type,geometry,properties
0,8341428,Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[3....",{'idmutinvar': '83a68c1f0c617ae2609e8ade54c118...


On a importé un fichier geojson mais qu'on a traité comme un json classique ; le code ci-dessous vise à mieux prendre en compte cette dimension spatiale (et le côté "geo" du fichier geojson)

In [74]:
mut_gdf = gpd.read_file(StringIO(response.text))

# Afficher un aperçu des données
print(mut_gdf.head())

Skipping field l_codinsee: unsupported OGR type: 5
Skipping field l_idpar: unsupported OGR type: 5
Skipping field l_idparmut: unsupported OGR type: 5
Skipping field l_idlocmut: unsupported OGR type: 5


                         idmutinvar                        idopendata  \
0  83a68c1f0c617ae2609e8ade54c11804  83a68c1f0c617ae2609e8ade54c11804   
1  45005cd8bafee591ed6debb57bbc6613  45005cd8bafee591ed6debb57bbc6613   
2  ebbcec3f97add3b68e7b452f8ba73f21  ebbcec3f97add3b68e7b452f8ba73f21   
3  7bcb49cdc14b9303042d3938d9ff7510  7bcb49cdc14b9303042d3938d9ff7510   
4  aee36b8e79a346304326d5bbed32e4f5  aee36b8e79a346304326d5bbed32e4f5   

     datemut  anneemut coddep                           libnatmut   vefa  \
0 2020-09-10      2020     59                               Vente  False   
1 2020-12-23      2020     59  Vente en l'état futur d'achèvement   True   
2 2020-12-17      2020     59                               Vente  False   
3 2020-07-07      2020     59                               Vente  False   
4 2020-12-04      2020     59                               Vente  False   

  valeurfonc  nbcomm  nbpar  nbparmut sterr  nbvolmut  nblocmut  sbati  \
0  200000.00       1      1   

Ok, mieux, mais il faudrait comprendre et régler ce qui s'affiche juste au dessus "Skipping field l_codinsee: unsupported OGR type: 5" (et les autres erreurs du même type)

In [75]:
# Afficher les colonnes
print(mut_gdf.columns)

Index(['idmutinvar', 'idopendata', 'datemut', 'anneemut', 'coddep',
       'libnatmut', 'vefa', 'valeurfonc', 'nbcomm', 'nbpar', 'nbparmut',
       'sterr', 'nbvolmut', 'nblocmut', 'sbati', 'codtypbien', 'libtypbien',
       'geometry'],
      dtype='object')


In [76]:
mut_gdf.head()

,idmutinvar,idopendata,datemut,anneemut,coddep,libnatmut,vefa,valeurfonc,nbcomm,nbpar,nbparmut,sterr,nbvolmut,nblocmut,sbati,codtypbien,libtypbien,geometry
0,83a68c1f0c617ae2609e8ade54c11804,83a68c1f0c617ae2609e8ade54c11804,2020-09-10,2020,59,Vente,False,200000.00,1,1,0,0.00,0,1,37.00,121,UN APPARTEMENT,"MULTIPOLYGON (((3.05323 50.6372, 3.05317 50.63..."
1,45005cd8bafee591ed6debb57bbc6613,45005cd8bafee591ed6debb57bbc6613,2020-12-23,2020,59,Vente en l'état futur d'achèvement,True,238800.02,1,1,0,0.00,0,0,0.00,101,BATI - INDETERMINE : Vefa sans descriptif,"MULTIPOLYGON (((3.06685 50.62606, 3.06694 50.6..."
2,ebbcec3f97add3b68e7b452f8ba73f21,ebbcec3f97add3b68e7b452f8ba73f21,2020-12-17,2020,59,Vente,False,294000.00,1,1,0,0.00,0,1,40.00,121,UN APPARTEMENT,"MULTIPOLYGON (((3.05946 50.64373, 3.05957 50.6..."
3,7bcb49cdc14b9303042d3938d9ff7510,7bcb49cdc14b9303042d3938d9ff7510,2020-07-07,2020,59,Vente,False,1.00,1,1,0,0.00,0,1,54.00,121,UN APPARTEMENT,"MULTIPOLYGON (((3.05761 50.6304, 3.05759 50.63..."
4,aee36b8e79a346304326d5bbed32e4f5,aee36b8e79a346304326d5bbed32e4f5,2020-12-04,2020,59,Vente,False,4500.00,1,1,0,0.00,0,1,0.00,131,UNE DEPENDANCE,"MULTIPOLYGON (((3.0738 50.61993, 3.07384 50.61..."


Description des variables de ce dataframe :
- idmutinvar = identifiant unique entre les diff bases pour retrouver chaque mutation
- idopendata = identifiant de mutation utilisé depuis l'ouverture de données DVF
- datemut = date de signature de l'acte de vente
- anneemut = annee de signature du doc
- codedep
- libnatmut = Libellé de nature de mutation tel que défini dans la donnée source DVF 6 catégories : Vente, Vente en l'état futur d'achèvement, Adjudication, Echange, Vente terrain à bâtir et Expropriation.
- vefa = booléen selon si la vente est en VEFA (vente en l'état futur d'achèvement) ou non
- valeurfonc = valeur foncière en euro déclarée lors de la vente ; prix net vendeur, inclut la TVA mais n'inclut pas les frais de notaire, d'agence ou des meubles. 
- nbcomm = nombre de communes concernée par la mutation (i.e. sur lesquelles se trouve la parcelle cadastrale concernée par la mutation)
- nbpar = comme nbcomm mais pour les parcelles.
- nbparmut = nb de parcelles ayant muté (i.e. dont la pleine propriété a été transférée lors de la mutation)
- sterr = surface de terrain ayant muté (mutation = transfert de propriété, donc surface de terrain ayant muté = surface du terrain qui a été vendu)
- nbvolmut = variable obscure, flemme 
- nblocmut = nb de locaux ayant muté
- sbati = surface de l'ensemble du bati ayant muté
- codtypbien = typologie des biens ; on pourra s'en servir pour filter les maisons et appartements et virer tous les types de batiments bizarres (cf schéma ici) https://doc-datafoncier.cerema.fr/doc/dv3f/mutation/codtypbien
- libtypbien = comme codtypbien mais pour la base DV3F et non issu de juste DVF
- geometry

In [77]:
len(mut_gdf)

100

In [78]:
maxi = 0
mini = 10000000000000000000000000000000000000
for index, row in mut_gdf.iterrows():
    if float(row["anneemut"]) > maxi : 
        maxi = float(row["anneemut"])
    if float(row["anneemut"]) < mini : 
        mini = float(row["anneemut"])
print(mini, maxi)

2020.0 2020.0


On est donc capable de trouver des valeurs qui ont l'air pas trop extrêmes pour la valeur de la transaction. 

In [79]:
mut_gdf_verif_vente = mut_gdf[mut_gdf["libnatmut"] != "Vente"]
len(mut_gdf_verif_vente)
    

14

Toutes les observations sont des ventes donc pas de problème de ce côté. Par ailleurs, on remarque qu'on peut travailler sur des colonnes de types object sans changer ce type s'il s'agit de texte (on n'a pas besoin de les transformer en string).

Mais on a un problème avec la surface transférée : parfois on voit 0 pour le terrain vendu et pour le bâti vendu (respectivement sterr et sbati). Il faut écarter ces anomalies. 

In [80]:
mut_gdf.dtypes

idmutinvar            object
idopendata            object
datemut       datetime64[ms]
anneemut               int32
coddep                object
libnatmut             object
vefa                    bool
valeurfonc            object
nbcomm                 int32
nbpar                  int32
nbparmut               int32
sterr                 object
nbvolmut               int32
nblocmut               int32
sbati                 object
codtypbien            object
libtypbien            object
geometry            geometry
dtype: object

In [81]:
mut_gdf['sterr']  = mut_gdf["sterr"].astype(float)
mut_gdf['sbati']  = mut_gdf["sbati"].astype(float)

In [82]:
mut_gdf.dtypes

idmutinvar            object
idopendata            object
datemut       datetime64[ms]
anneemut               int32
coddep                object
libnatmut             object
vefa                    bool
valeurfonc            object
nbcomm                 int32
nbpar                  int32
nbparmut               int32
sterr                float64
nbvolmut               int32
nblocmut               int32
sbati                float64
codtypbien            object
libtypbien            object
geometry            geometry
dtype: object

In [83]:
mut_gdf_ano = mut_gdf[mut_gdf["sterr"]==0]
len(mut_gdf_ano)


70

In [84]:
mut_gdf_anomalie = mut_gdf_ano[mut_gdf_ano["sbati"]==0]
len(mut_gdf_anomalie)

12

In [85]:
mut_gdf_filtre = mut_gdf[mut_gdf['sterr'] != 0 or mut_gdf['sbati'] != 0]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().